In [1]:
%load_ext autoreload
%autoreload 2
%load_ext nb_black
%load_ext lab_black

<IPython.core.display.Javascript object>

In [2]:
# default_exp submission

<IPython.core.display.Javascript object>

# Submission

These objects allow you to easily and reliably submit predictions for Numerai Classic and Numerai Signals.

The main objects are:
1. `NumeraiClassicSubmittor`
2. `NumeraiSignalsSubmittor`

In [3]:
# hide
from nbdev.showdoc import *

<IPython.core.display.Javascript object>

In [4]:
# export
import os
import uuid
import numpy as np
import pandas as pd
from typing import Union
from copy import deepcopy
from random import choices
from tqdm.auto import tqdm
from datetime import datetime
from abc import abstractmethod
from typeguard import typechecked
from string import ascii_uppercase
from rich import print as rich_print
from numerapi import NumerAPI, SignalsAPI
from dateutil.relativedelta import relativedelta, FR

from numerai_blocks.download import BaseIO
from numerai_blocks.key import Key

<IPython.core.display.Javascript object>

## 1. BaseSubmittor

`BaseSubmittor` handles all submission logic common to Numerai Classic and Numerai Signals. Under the hood directory logic is handled by `BaseIO`.
Each submittor should inherit from `BaseSubmittor` and implement the `.save_csv` method.

In [5]:
# export
@typechecked
class BaseSubmittor(BaseIO):
    def __init__(self, directory_path: str, api: Union[NumerAPI, SignalsAPI]):
        super().__init__(directory_path)
        self.api = api

    @abstractmethod
    def save_csv(
        self,
        dataf: pd.DataFrame,
        file_name: str,
        cols: Union[str, list],
        *args,
        **kwargs,
    ):
        """
        For Numerai Classic: Save index column + 'cols' (targets) to CSV.
        For Numerai Signals: Save ticker, friday_date, data_type and signal columns to CSV.
        """
        ...

    def upload_predictions(self, file_name: str, model_name: str, *args, **kwargs):
        """
        Upload CSV file to Numerai for given model name.
        :param file_name: File name/path relative to directory_path.
        :param model_name: Lowercase raw model name (For example, 'integration_test').
        """
        full_path = str(self.dir / file_name)
        model_id = self._get_model_id(model_name=model_name)
        api_type = str(self.api.__class__.__name__)
        rich_print(
            f":airplane: {api_type}: Uploading predictions from '{full_path}' for model [bold blue]'{model_name}'[/bold blue] (model_id='{model_id}') :airplane:"
        )
        self.api.upload_predictions(
            file_path=full_path, model_id=model_id, *args, **kwargs
        )
        rich_print(
            f":thumbs_up: {api_type} submission of '{full_path}' for [bold blue]{model_name}[/bold blue] is successful! :thumbs_up:"
        )

    def full_submission(
        self,
        dataf: pd.DataFrame,
        file_name: str,
        model_name: str,
        cols: Union[str, list],
        *args,
        **kwargs,
    ):
        """
        Save DataFrame to csv and upload predictions through API.
        *args, **kwargs are passed to numerapi API.
        """
        self.save_csv(dataf=dataf, file_name=file_name, cols=cols)
        self.upload_predictions(
            file_name=file_name, model_name=model_name, *args, **kwargs
        )

    def _get_model_id(self, model_name: str) -> str:
        """
        Get ID needed for prediction uploading.
        :param model_name: Raw lowercase model name
        of model that you have access to.
        """
        return self.get_model_mapping[model_name]

    @property
    def get_model_mapping(self) -> dict:
        """Mapping between raw model names and model IDs."""
        return self.api.get_models()

    def _check_value_range(self, dataf: pd.DataFrame, cols: Union[str, list]):
        """ Check if all predictions are in range (0...1). """
        cols = [cols] if isinstance(cols, str) else cols
        for col in cols:
            if not dataf[col].between(0, 1).all():
                min_val, max_val = dataf[col].min(), dataf[col].max()
                raise ValueError(
                    f"Values in 'signal' must be between 0 and 1 (exclusive). \
Found min value of '{min_val}' and max value of '{max_val}' for column '{col}'."
                )

    def __call__(
            self,
            dataf: pd.DataFrame,
            file_name: str,
            model_name: str,
            cols: Union[str, list],
            *args,
            **kwargs,
    ):
        """
        The most common use case will be to create a CSV and submit it immediately after that.
        full_submission handles this.
        """
        self.full_submission(
            dataf=dataf,
            file_name=file_name,
            model_name=model_name,
            cols=cols,
            *args,
            **kwargs,
        )


<IPython.core.display.Javascript object>

## 2. Numerai Classic

For Numerai Classic submissions. Uses [NumerAPI](https://numerapi.readthedocs.io/en/latest/_modules/numerapi/numerapi.html) under the hood.

Note that using submittors requires a `Key` object. For more information about this, see section 8.

In [6]:
# export
@typechecked
class NumeraiClassicSubmittor(BaseSubmittor):
    """
    Submit for Numerai Classic.
    :param directory_path: Base directory to save and read prediction files from.
    :param key: Key object (numerai-blocks.key.Key) containing valid credentials for Numerai Classic.
    *args, **kwargs will be passed to NumerAPI initialization.
    """
    def __init__(self, directory_path: str, key: Key, *args, **kwargs):
        api = NumerAPI(public_id=key.pub_id, secret_key=key.secret_key, *args, **kwargs)
        super().__init__(
            directory_path=directory_path, api=api
        )

    def save_csv(
        self,
        dataf: pd.DataFrame,
        file_name: str,
        cols: Union[str, list] = "prediction",
        *args,
        **kwargs,
    ):
        """
        :param dataf: DataFrame which should have at least the following columns:
        1. id (as index column)
        2. cols (for example, 'prediction', ['prediction'] or [ALL_NUMERAI_TARGETS]).
        :param file_name: .csv file path .
        :param cols: All prediction columns.
        For example, 'prediction', ['prediction'] or [ALL_NUMERAI_TARGETS].
        """
        self._check_value_range(dataf=dataf, cols=cols)

        full_path = str(self.dir / file_name)
        rich_print(
            f":page_facing_up: Saving predictions CSV to '{full_path}'. :page_facing_up:"
        )
        dataf.loc[:, cols].to_csv(full_path, *args, **kwargs)

    def combine_csvs(self, csv_paths: list) -> pd.DataFrame:
        """
        Read in csv files and combine all predictions with a rank mean.
        Multi-target predictions will be averaged out.
        :param csv_paths: List of full paths to .csv prediction files.
        """
        datafs = []
        for i, path in tqdm(enumerate(csv_paths),
                            desc=f"Combine CSVs"):
            dataf = pd.read_csv(path, index_col="id")
            datafs.append(dataf)
        final_dataf = pd.concat(datafs, axis="columns")
        final_dataf['prediction'] = final_dataf.rank(pct=True, method="first").mean(axis=1)
        return final_dataf.loc[:, ['prediction']]

<IPython.core.display.Javascript object>

### NumeraiClassicSubmittor tests

In [7]:
# Initialization (Random credentials)
test_dir = "test_sub"
classic_key = Key(pub_id="UFVCTElDX0lE", secret_key="U1VQRVJfU0VDUkVUX0tFWQ==")
num_sub = NumeraiClassicSubmittor(directory_path=test_dir, key=classic_key)
assert num_sub.dir.is_dir()

# Create random dataframe
n_rows, n_columns = 100, 20
targets = [f"prediction_{i}" for i in range(n_columns)]
test_dataf = pd.DataFrame(np.random.uniform(size=(n_rows, n_columns)), columns=targets)
test_dataf["id"] = [uuid.uuid4() for _ in range(n_rows)]
test_dataf = test_dataf.set_index("id")
test_dataf.head(2)

🔑 Numerai Auth key initialized with pub_id = 'UFVCTElDX0lE' 🔑

,prediction_0,prediction_1,prediction_2,prediction_3,prediction_4,prediction_5,prediction_6,prediction_7,prediction_8,prediction_9,prediction_10,prediction_11,prediction_12,prediction_13,prediction_14,prediction_15,prediction_16,prediction_17,prediction_18,prediction_19
id,,,,,,,,,,,,,,,,,,,,
1e27ec49-3f5d-4b3f-9ec9-5f3f79626360,0.910276,0.893424,0.052896,0.079084,0.142604,0.209662,0.668440,0.55767,0.524363,0.948481,0.284003,0.645700,0.540471,0.330153,0.238646,0.610122,0.005369,0.222632,0.304998,0.523189
c4fd64cf-3ffd-4e63-a1c7-22cddfc7f185,0.125541,0.562856,0.848698,0.759123,0.247746,0.608131,0.022391,0.51516,0.588931,0.324228,0.671961,0.619421,0.116497,0.954820,0.949490,0.819323,0.368915,0.794054,0.299073,0.829131


<IPython.core.display.Javascript object>

In [8]:
file_name = "test.csv"
num_sub.save_csv(dataf=test_dataf, file_name=file_name, cols=targets)
num_sub.save_csv(dataf=test_dataf, file_name="test2.csv", cols=targets)
pd.read_csv(f"{test_dir}/{file_name}").head(2)

2022-02-21 12:53:21,513 INFO numexpr.utils: Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2022-02-21 12:53:21,513 INFO numexpr.utils: NumExpr defaulting to 8 threads.


📄 Saving predictions CSV to 'test_sub/test.csv'. 📄

📄 Saving predictions CSV to 'test_sub/test2.csv'. 📄

,id,prediction_0,prediction_1,prediction_2,prediction_3,prediction_4,prediction_5,prediction_6,prediction_7,prediction_8,...,prediction_10,prediction_11,prediction_12,prediction_13,prediction_14,prediction_15,prediction_16,prediction_17,prediction_18,prediction_19
0,1e27ec49-3f5d-4b3f-9ec9-5f3f79626360,0.910276,0.893424,0.052896,0.079084,0.142604,0.209662,0.668440,0.55767,0.524363,...,0.284003,0.645700,0.540471,0.330153,0.238646,0.610122,0.005369,0.222632,0.304998,0.523189
1,c4fd64cf-3ffd-4e63-a1c7-22cddfc7f185,0.125541,0.562856,0.848698,0.759123,0.247746,0.608131,0.022391,0.51516,0.588931,...,0.671961,0.619421,0.116497,0.954820,0.949490,0.819323,0.368915,0.794054,0.299073,0.829131


<IPython.core.display.Javascript object>

In [9]:
combined = num_sub.combine_csvs(["test_sub/test.csv", "test_sub/test2.csv"])
assert combined.columns == ['prediction']
combined.head(2)

Combine CSVs: 0it [00:00, ?it/s]

,prediction
id,
1e27ec49-3f5d-4b3f-9ec9-5f3f79626360,0.4420
c4fd64cf-3ffd-4e63-a1c7-22cddfc7f185,0.5605


<IPython.core.display.Javascript object>

In [10]:
def test_signal_validity(
        submittor: NumeraiClassicSubmittor, dataf: pd.DataFrame
):
    """ Test value range of prediction. """
    try:
        invalid_signal = deepcopy(dataf)
        invalid_signal.iloc[0]["prediction_0"] += 10
        submittor.save_csv(
            invalid_signal,
            file_name="should_not_save.csv",
            cols=list(invalid_signal.columns),
        )
    except ValueError:
        return True
    return False

assert test_signal_validity(num_sub, test_dataf)

<IPython.core.display.Javascript object>

In [11]:
# Uncomment to save CSV and upload predictions
# num_sub.full_submission(dataf=test_dataf, file_name='test.csv', cols=targets, model_name="test")

<IPython.core.display.Javascript object>

In [12]:
# Remove contents after submitting is successful
num_sub.remove_base_directory()
assert not os.path.exists(test_dir)

⚠ Deleting directory for 'NumeraiClassicSubmittor' ⚠
Path: '/Users/clepelaars/Desktop/crowdcent/repositories/numerai-blocks/nbs/test_sub'

<IPython.core.display.Javascript object>

## 3. Numerai Signals

For Numerai Signals submissions. Uses [SignalsAPI](https://numerapi.readthedocs.io/en/latest/_modules/numerapi/signalsapi.html) under the hood.

In [13]:
# export
@typechecked
class NumeraiSignalsSubmittor(BaseSubmittor):
    """
    Submit for Numerai Signals
    :param directory_path: Base directory to save and read prediction files from.
    :param key: Key object (numerai-blocks.key.Key) containing valid credentials for Numerai Signals.
    *args, **kwargs will be passed to SignalsAPI initialization.
    """

    def __init__(self, directory_path: str, key: Key, *args, **kwargs):
        api = SignalsAPI(
            public_id=key.pub_id, secret_key=key.secret_key, *args, **kwargs
        )
        super().__init__(
            directory_path=directory_path, api=api
        )
        self.supported_ticker_formats = [
            "cusip",
            "sedol",
            "ticker",
            "numerai_ticker",
            "bloomberg_ticker",
        ]

    def save_csv(
        self, dataf: pd.DataFrame, file_name: str, cols: list = None, *args, **kwargs
    ):
        """
        :param dataf: DataFrame which should have at least the following columns:
         1. One of supported ticker formats (cusip, sedol, ticker, numerai_ticker or bloomberg_ticker)
         2. signal (Values between 0 and 1 (exclusive))
         Additional columns for if you include validation data (optional):
         3. friday_date (YYYYMMDD format date indication)
         4. data_type ('val' and 'live' partitions)

         :param file_name: .csv file path.
         :param cols: All cols that should be passed to CSV. Defaults to 2 standard columns.
          ('bloomberg_ticker', 'signal')
        """
        if not cols:
            cols = ["bloomberg_ticker", "signal"]

        self._check_ticker_format(cols=cols)
        self._check_value_range(dataf=dataf, cols="signal")

        full_path = str(self.dir / file_name)
        rich_print(
            f":page_facing_up: Saving Signals predictions CSV to '{full_path}'. :page_facing_up:"
        )
        dataf.loc[:, cols].reset_index(drop=True).to_csv(
            full_path, index=False, *args, **kwargs
        )

    def _check_ticker_format(self, cols: list):
        """ Check for valid ticker format. """
        valid_tickers = set(cols).intersection(set(self.supported_ticker_formats))
        if not valid_tickers:
            raise NotImplementedError(
                f"No supported ticker format in {cols}). \
Supported: '{self.supported_ticker_formats}'"
            )


<IPython.core.display.Javascript object>

### NumeraiSignalsSubmittor tests

In [14]:
# Initialization (Random credentials)
test_dir_signals = "test_sub_signals"
signals_key = Key(pub_id="UFVCTElDX0lE", secret_key="U1VQRVJfU0VDUkVUX0tFWQ==")
signals_sub = NumeraiSignalsSubmittor(directory_path=test_dir_signals, key=signals_key)
assert signals_sub.dir.is_dir()

🔑 Numerai Auth key initialized with pub_id = 'UFVCTElDX0lE' 🔑

No existing directory found at 'test_sub_signals'. Creating directory...

<IPython.core.display.Javascript object>

In [15]:
# Create random dataframe
n_rows, n_columns = 5000, 4
signals_test_dataf = pd.DataFrame(
    np.random.uniform(size=(n_rows, 1)), columns=["signal"]
)
signals_test_dataf["ticker"] = [
    "".join(choices(ascii_uppercase, k=4)) for _ in range(n_rows)
]
last_friday = int(
    str((datetime.now() + relativedelta(weekday=FR(-1))).date()).replace("-", "")
)
signals_test_dataf["friday_date"] = last_friday
signals_test_dataf["data_type"] = "live"
signals_test_dataf["aux_column"] = "metadata"
signals_test_dataf.head(2)

,signal,ticker,friday_date,data_type,aux_column
0,0.424141,MMTX,20220218,live,metadata
1,0.972631,JWMS,20220218,live,metadata


<IPython.core.display.Javascript object>

In [16]:
signals_cols = ["signal", "ticker", "friday_date", "data_type"]
file_name = "signals_test.csv"
signals_sub.save_csv(dataf=signals_test_dataf, file_name=file_name, cols=signals_cols)
pd.read_csv(f"{test_dir_signals}/{file_name}").head(2)

📄 Saving Signals predictions CSV to 'test_sub_signals/signals_test.csv'. 📄

,signal,ticker,friday_date,data_type
0,0.424141,MMTX,20220218,live
1,0.972631,JWMS,20220218,live


<IPython.core.display.Javascript object>

Saving Signals CSV should fail if there is no valid ticker column or if `signal` has values outside the range $(0...1)$.

In [17]:
def test_signal_validity(
    submittor: NumeraiSignalsSubmittor, signals_dataf: pd.DataFrame
):
    """ Test value range of signal. """
    try:
        invalid_signal = deepcopy(signals_dataf)
        invalid_signal.loc[0, "signal"] += 10
        submittor.save_csv(
            invalid_signal,
            file_name="should_not_save.csv",
            cols=list(invalid_signal.columns),
        )
    except ValueError:
        return True
    return False


def test_ticker_validity(
    submittor: NumeraiSignalsSubmittor, signals_dataf: pd.DataFrame
):
    """ Test safeguard if ticker column is not valid. """
    try:
        invalid_ticker = deepcopy(signals_dataf)
        invalid_ticker = invalid_ticker.rename(
            {"ticker": "not_a_valid_ticker_format"}, axis=1
        )
        submittor.save_csv(
            invalid_ticker,
            file_name="should_not_save.csv",
            cols=list(invalid_ticker.columns),
        )
    except NotImplementedError:
        return True
    return False

assert test_signal_validity(signals_sub, signals_test_dataf)
assert test_ticker_validity(signals_sub, signals_test_dataf)

<IPython.core.display.Javascript object>

In [18]:
# Uncomment to save CSV and upload predictions
# signals_sub.full_submission(dataf=signals_test_dataf, file_name='signals_test.csv', cols=signals_cols, model_name="test")

<IPython.core.display.Javascript object>

In [19]:
# Remove contents after submission is successful
signals_sub.remove_base_directory()
assert not os.path.exists(test_dir_signals)

⚠ Deleting directory for 'NumeraiSignalsSubmittor' ⚠
Path: '/Users/clepelaars/Desktop/crowdcent/repositories/numerai-blocks/nbs/test_sub_signals'

<IPython.core.display.Javascript object>

------------------------------------------------------------

In [20]:
# hide
# Run this cell to sync all changes with library
from nbdev.export import notebook2script

notebook2script()

Converted 00_misc.ipynb.
Converted 01_download.ipynb.
Converted 02_numerframe.ipynb.
Converted 03_preprocessing.ipynb.
Converted 04_model.ipynb.
Converted 05_postprocessing.ipynb.
Converted 06_modelpipeline.ipynb.
Converted 07_evaluation.ipynb.
Converted 08_key.ipynb.
Converted 09_submission.ipynb.
Converted 10_staking.ipynb.
Converted index.ipynb.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>